In [49]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv


In [50]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [51]:
df.drop(["RowNumber", "Surname", "CustomerId"],axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [52]:
df.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [53]:
df.replace({"Female":0, "Male":1}, inplace = True)
df = pd.get_dummies(data = df, columns = ["Geography","Tenure","NumOfProducts"], drop_first = True)

In [54]:
df.isna().sum()

CreditScore          0
Gender               0
Age                  0
Balance              0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_Germany    0
Geography_Spain      0
Tenure_1             0
Tenure_2             0
Tenure_3             0
Tenure_4             0
Tenure_5             0
Tenure_6             0
Tenure_7             0
Tenure_8             0
Tenure_9             0
Tenure_10            0
NumOfProducts_2      0
NumOfProducts_3      0
NumOfProducts_4      0
dtype: int64

In [55]:
df.head()

,CreditScore,Gender,Age,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,...,Tenure_4,Tenure_5,Tenure_6,Tenure_7,Tenure_8,Tenure_9,Tenure_10,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
0,619,0,42,0.00,1,1,101348.88,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,608,0,41,83807.86,0,1,112542.58,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,502,0,42,159660.80,1,0,113931.57,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3,699,0,39,0.00,0,0,93826.63,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,850,0,43,125510.82,1,1,79084.10,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [56]:
cols_to_scale = ["CreditScore", "Age", "Balance", "EstimatedSalary"]

In [57]:
## normalizing the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
df.head()

,CreditScore,Gender,Age,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,...,Tenure_4,Tenure_5,Tenure_6,Tenure_7,Tenure_8,Tenure_9,Tenure_10,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
0,0.538,0,0.324324,0.000000,1,1,0.506735,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.516,0,0.310811,0.334031,0,1,0.562709,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.304,0,0.324324,0.636357,1,0,0.569654,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0.698,0,0.283784,0.000000,0,0,0.469120,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1.000,0,0.337838,0.500246,1,1,0.395400,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [58]:
X = df.drop("Exited", axis = 1)
y = df.Exited

In [59]:
y.value_counts(normalize = True)*100

0    79.63
1    20.37
Name: Exited, dtype: float64

## start modelling without balancing

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, shuffle = True, random_state = 1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7500, 22), (2500, 22), (7500,), (2500,))

In [62]:
from tensorflow_addons import losses
from tensorflow.keras import models, layers, Sequential, callbacks, initializers

In [63]:
def ANN(X_train, y_train, X_test, y_test, loss, weights, epochs):
    dim = X_train.shape[1]
    model = Sequential([
        layers.Dense(30,input_dim = dim, activation = "relu"),
        layers.Dense(60, activation = "relu"),
        layers.Dense(50, activation = "relu"),
        layers.Dense(30, activation = "relu"),
        layers.Dense(1, activation = "sigmoid")
        
    ])
    model.compile(optimizer = "adam", loss = loss, metrics = ["accuracy"])
    if weights == 1:
        model.fit(X_train, y_train, epochs = epochs)
    else:
        model.fit(X_train, y_train, epochs = epochs, class_weight = weights)
        
    print(model.evaluate(X_test, y_test))
    
    preds = model.predict(X_test)
    preds = np.round(preds)
    report = classification_report(y_test, preds)
    
    print(report)
    return preds

In [64]:
preds1 = ANN(X_train, y_train, X_test, y_test, "binary_crossentropy", epochs = 100, weights = 1)

Epoch 1/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4373 - accuracy: 0.8103
Epoch 2/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3892 - accuracy: 0.8357
Epoch 3/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3731 - accuracy: 0.8439
Epoch 4/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3641 - accuracy: 0.8496
Epoch 5/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3549 - accuracy: 0.8552
Epoch 6/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3485 - accuracy: 0.8576
Epoch 7/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3431 - accuracy: 0.8571
Epoch 8/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3374 - accuracy: 0.8625
Epoch 9/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3310 - accuracy: 0.8639
Epoch 10/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3283 - accura

In [65]:
## this gave us f1-score of: 0.49 for the minority class and 0.88 for majority class
## accuracy 0.81

## using undersampling

In [66]:
class0, class1 = df.Exited.value_counts()
df_0 = df[df.Exited == 0]
df_1 = df[df.Exited == 1]

In [67]:
df_0_under = df_0.sample(class1)
df_undersampled = pd.concat([df_0_under, df_1], axis = 0)
df_undersampled.Exited.value_counts()

1    2037
0    2037
Name: Exited, dtype: int64

In [68]:
X = df_undersampled.drop("Exited", axis = 1)
y = df_undersampled.Exited

In [69]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify = y, shuffle = True)
preds2 = ANN(X2_train, y2_train, X2_test, y2_test,"binary_crossentropy", 1, 100)

Epoch 1/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6296 - accuracy: 0.6585
Epoch 2/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5497 - accuracy: 0.7211
Epoch 3/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5302 - accuracy: 0.7272
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5192 - accuracy: 0.7456
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5122 - accuracy: 0.7469
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4990 - accuracy: 0.7539
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7561
Epoch 8/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4848 - accuracy: 0.7588
Epoch 9/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4785 - accuracy: 0.7647
Epoch 10/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4731 - accura

In [70]:
## undersampling gave us an f1-score for both the minority and the majority class of 0.72 
## accuracy 0.72

## using oversampling

In [71]:
df_1_over = df_1.sample(class0, replace = True)
df_oversampled = pd.concat([df_0, df_1_over], axis = 0)
df_oversampled.Exited.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [72]:
X = df_oversampled.drop("Exited", axis = 1)
y = df_oversampled.Exited

In [73]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify = y, shuffle = True)
preds3 = ANN(X3_train, y3_train, X3_test, y3_test,"binary_crossentropy", 1, 100)

Epoch 1/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5491 - accuracy: 0.7206
Epoch 2/100
399/399 [==============================] - 1s 3ms/step - loss: 0.4979 - accuracy: 0.7580
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4839 - accuracy: 0.7651
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4689 - accuracy: 0.7760
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4602 - accuracy: 0.7808
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4510 - accuracy: 0.7842
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4407 - accuracy: 0.7930
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4369 - accuracy: 0.7931
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4242 - accuracy: 0.8019
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4123 - accura

In [74]:
## best f1-score so far: 0.89 for majority class and 0.90 for minority
## accuracy 0.89

## trying SMOTE

In [75]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy = "minority")
X = df.drop("Exited", axis = 1)
y = df.Exited
X_sm, y_sm = smote.fit_sample(X, y)
y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [76]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X_sm, y_sm, test_size = 0.2, random_state = 1, stratify = y_sm, shuffle = True)
preds4 = ANN(X4_train, y4_train, X4_test, y4_test,"binary_crossentropy", 1, 100)

Epoch 1/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5304 - accuracy: 0.7334
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4709 - accuracy: 0.7779
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4540 - accuracy: 0.7849
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4428 - accuracy: 0.7931
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4335 - accuracy: 0.7980
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4247 - accuracy: 0.8018
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4146 - accuracy: 0.8084
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4046 - accuracy: 0.8136
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.3965 - accuracy: 0.8208
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.3861 - accura

In [77]:
## f1-score for majority is 0.85 and for minority 0.86
# accuracy 0.85

## Conclusion:
by only using these 4 methods, Oversampling produced the best F1 Score!

# # Trying with tensorflow tutorial

In [78]:
X = df.drop("Exited", axis = 1)
y = df.Exited

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, shuffle = True, random_state = 1)

In [106]:
import tensorflow_addons as tfa


f1 = tfa.metrics.F1Score(num_classes = 2, average = None,threshold = 0.5)

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

def make_model(output_bias=None):
  if output_bias is not None:
    output_bias = initializers.Constant(output_bias)
  model = Sequential([
      layers.Dense(
          16, activation='relu',
          input_shape=(X_train.shape[1],)),
      layers.Dropout(0.5),
      layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer="adam",
      loss="binary_crossentropy",
      metrics=[METRICS])

  return model

In [107]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


In [108]:
model = make_model()
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 16)                368       
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 17        
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________


In [109]:
preds = model.predict(X_train)
results = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
print(results)

4/4 [==============================] - 0s 6ms/step - loss: 0.5188 - tp: 7.0000 - fp: 45.0000 - tn: 6325.0000 - fn: 1623.0000 - accuracy: 0.7915 - precision: 0.1346 - recall: 0.0043 - auc: 0.5397
[0.5188287496566772, 7.0, 45.0, 6325.0, 1623.0, 0.7914999723434448, 0.13461539149284363, 0.0042944783344864845, 0.5396704077720642]


F-1 = 2 * (precision * recall) / (precision + recall)

In [112]:
f1 = 2*(0.1346*0.0043)/(0.1346+0.0043)
f1

0.008333765298776098

In [110]:
initial_bias = np.log([class1/class0])
initial_bias

array([-1.36332768])

In [111]:
model = make_model(output_bias = initial_bias)
preds = model.predict(X_train)
results = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
print(results)

4/4 [==============================] - 0s 5ms/step - loss: 0.5384 - tp: 7.0000 - fp: 45.0000 - tn: 12695.0000 - fn: 3253.0000 - accuracy: 0.7939 - precision: 0.1346 - recall: 0.0021 - auc: 0.5446
[0.5383965969085693, 7.0, 45.0, 12695.0, 3253.0, 0.793874979019165, 0.13461539149284363, 0.0021472391672432423, 0.5445557832717896]


In [ ]:
f1 = 2*(0.1346*0.0021)/(0.1346+0.0021)
f1